<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2025notebooks/2025_0604ROC_AUC_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ROC, AUC 曲線実習

* sepal (萼片 がくべん)： 花の外側の部分 (多くの場合緑色で葉のようなもの) で発達中の蕾を包む。
* petal (花弁，花びら)：花の外側の部分で，しばしば目立つ色をしている。

<img src="https://dictionary.goo.ne.jp/img/daijisen/ref/113205.jpg"><br/>
Goo 国語辞典より



# HAD データの取得


In [ ]:
# HAD データの取得

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install --upgrade xlrd
    !pip install --upgrade pandas

import os
import pandas as pd
had_fname = 'HAD_sample_data.xls'
if not os.path.exists(had_fname):
    # HAD サンプルデータをダウンロード
    !wget 'https://files.au-1.osf.io/v1/resources/32cyp/providers/osfstorage/5fb340145502ac018d8c86ab/?zip=' -O had.zip
    !unzip had.zip

# HAD のサンプルデータ読み込み
df = pd.read_excel('HAD_sample_data.xls', sheet_name="iris", index_col='ID')
df

In [ ]:
# 上のセルが時間がかかる場合には scikit-learn のデータを使用
from sklearn.datasets import load_iris

df = pd.DataFrame(np.concat((np.array(load_iris().target).reshape(-1,1), np.array(load_iris().data)), axis=1),  columns=['Spices', 'Sepal.L','Sepal.W',	'Petal.L','Petal.W'])
#df = pd.DataFrame([load_iris().target, load_iris().data], columns=['Spices', 'Sepal.L','Sepal.W',	'Petal.L','Petal.W'])
#df['Spices'] = load_iris().target
df #type(load_iris().frame)

In [ ]:
print(load_iris().DESCR)

In [39]:
%matplotlib inline
# https://www.geeksforgeeks.org/how-to-plot-roc-curve-in-python/
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components='mle')
pca.fit(load_iris().data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

y = pca.transform(load_iris().data)
print(f'y.shape:{y.shape}')

plt.figure(figsize=(10,10))
plt.scatter(y[:,0], y[:,1], s=1)
for i, x in enumerate(load_iris().target):
    plt.annotate(load_iris().target_names[x], (y[i,0], y[i,1]), fontsize=10)
plt.show()
print(load_iris().target_names)

In [41]:
# 上の PCA 結果から versicolor を判別するのが難しそうである。
# なので 3 種のアヤメの中から versicolor を 1 とし 他 2 種類を 0 とする教師信号を作成する
tgt_virgicolor = [1 if tgt == 1 else 0 for tgt in load_iris().target]

# 3 種のアヤメの中から virginica を 1 とし 他 2 種類を 0 とする教師信号を作成する
tgt_virginica = [1 if tgt == 2 else 0 for tgt in load_iris().target]

# 3 種のアヤメの中から setosa を 1 とし 他 2 種類を 0 とする教師信号を作成する
tgt_setosa = [1 if tgt == 0 else 0 for tgt in load_iris().target]

X = load_iris().data

X_train, X_test, y_train, y_test = train_test_split(X, tgt_virgicolor, test_size=0.25, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, tgt_virginica, test_size=0.25, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, tgt_setosa, test_size=0.25, random_state=42)

# Train a logistic regression model
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

# Predict probabilities on the test set
y_pred_proba = model.predict_proba(X_test)[:, 1]

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label='ROC 曲線 (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('儀陽性率 FPR')
plt.ylabel('真陽性率 TPR')
plt.title('ROC 曲線')
plt.legend()
plt.show()

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd


# データの生成と分割
# `make_classification` を使って 80 対 20 の分割比率を用いて、20 次元の特徴を持つ人工的な 2 値分類データを作成
# このデータを訓練データセットと検証データセットに分割し、再現性を確保するために乱数の種を割り当てる。
X, y = make_classification(
    n_samples=1000, n_features=20, n_classes=2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# ランダム・フォレスト・モデルとロジスティック回帰モデルを訓練する
# コードを実行するたびに同じ結果が得られるように、固定のランダム・シードを使用。

# 訓練データを使ってロジスティック回帰モデルを訓練
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(X_train, y_train)

# 同じ訓練データとランダムシードを用いて、100本の木を持つランダムフォレスト・モデルを訓練する。
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)

In [44]:
# 検査データと訓練されたロジスティック回帰モデルを用いて、正のクラスの確率を予測。
y_pred_logistic = logistic_model.predict_proba(X_test)[:, 1]

# 同様に検査データを使い、訓練されたランダム・フォレスト・モデルを使って、正のクラスの予測確率を生成する。
y_pred_rf = random_forest_model.predict_proba(X_test)[:, 1]

In [ ]:
# テストデータを使って、「True」、『Logistic』、「RandomForest 」というラベルの付いた列を持つ test_df という DataFrame を作成
# 真のラベルとランダムフォレストとロジスティック回帰モデルからの予測確率を追加する。
test_df = pd.DataFrame(
    {'True': y_test, 'Logistic': y_pred_logistic, 'RandomForest': y_pred_rf})

# モデルの ROC 曲線をプロット
plt.figure(figsize=(7, 7))

for model in ['Logistic', 'RandomForest']:
    fpr, tpr, _ = roc_curve(test_df['True'], test_df[model])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{model} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'r--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Two Models')
plt.legend()
plt.show()

ランダム・フォレストとロジスティック回帰の AUC と ROC 曲線を計算し、ROC 曲線をプロットする。
ランダム推測の ROC 曲線も赤い破線で表現され、可視化のためにラベル、タイトル、凡例が設定される。
<!-- The plot computes the AUC and ROC curve for each model i.e Random Forest and Logistic Regression, then plots the ROC curve. The ROC curve for random guessing is also represented by a red dashed line, and labels, a title, and a legend are set for visualization.-->

多クラスモデルの ROC-AUC
複数クラス・モデルの場合、単純に1対すべての手法を使用することができ、各クラスについて1つのROC曲線が得られる。たとえば、A、B、C、Dの4つのクラスがあるとすると、4つのクラスすべてにROC曲線とそれに対応するAUC値が存在することになる。
<!-- ROC-AUC for a Multi-Class Model
For a multi-class model we can simply use one vs all methodology and you will have one ROC curve for each class. Let's say you have four classes A, B, C and D then there would be ROC curves and corresponding AUC values for all the four classes i.e once A would be one class and B, C and D combined would be the others class similarly B is one class and A, C and D combined as others class. -->

多クラス分類モデルの文脈でAUC-ROCを使用する一般的な手順は、次のとおりである：
<!-- The general steps for using AUC-ROC in the context of a multiclass classification model are: -->

**1 対全の手法**<br/>
* 多クラス問題の各クラスについて，それを正クラスとして扱い，他のすべてのクラスを負クラスに結合する．
* 残りのクラスに対して，各クラスのバイナリ分類器を訓練する．

**各クラスの AUC-ROC を計算**<br/>
* ここで，残りのクラスに対する任意のクラスのROC曲線をプロットする．
* 各クラスの ROC 曲線を同じグラフ上にプロットする．各曲線は、特定のクラスに対するモデルの識別性能を表す。
* 各クラスの AUC スコアを調べる．AUC スコアが高いほど、その特定のクラスについて識別性が高いことを示す。

多クラス分類での AUC-ROC の実装を見よう

<!-- One-vs-All Methodology:
For each class in your multiclass problem treat it as the positive class while combining all other classes into the negative class.
Train the binary classifier for each class against the rest of the classes.
Calculate AUC-ROC for Each Class:
Here we plot the ROC curve for the given class against the rest.
Plot the ROC curves for each class on the same graph. Each curve represents the discrimination performance of the model for a specific class.
Examine the AUC scores for each class. A higher AUC score indicates better discrimination for that particular class.
Lets see Implementation of AUC-ROC in Multiclass Classification -->


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from itertools import cycle

X, y = make_classification(
    n_samples=1000, n_features=20, n_classes=3, n_informative=10, random_state=42)

y_bin = label_binarize(y, classes=np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(
    X, y_bin, test_size=0.2, random_state=42)

logistic_model = OneVsRestClassifier(LogisticRegression(random_state=42))
logistic_model.fit(X_train, y_train)

rf_model = OneVsRestClassifier(
    RandomForestClassifier(n_estimators=100, random_state=42))
rf_model.fit(X_train, y_train)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

models = [logistic_model, rf_model]

plt.figure(figsize=(6, 5))
colors = cycle(['aqua', 'darkorange'])

for model, color in zip(models, colors):
    for i in range(model.classes_.shape[0]):
        fpr[i], tpr[i], _ = roc_curve(
            y_test[:, i], model.predict_proba(X_test)[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'{model.__class__.__name__} - Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve with Logistic Regression and Random Forest')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.stats import f_oneway

#load_iris().data, load_iris().target
print(load_iris().data[:,0])
_setosa     = [True if x == 0 else False for x in load_iris().target]
_vergicolor = [True if x == 1 else False for x in load_iris().target]
_verginica  = [True if x == 2 else False for x in load_iris().target]
_setosa_data = load_iris().data[_setosa,0]
_vergicolor_data = load_iris().data[_vergicolor,0]
_verginica_data = load_iris().data[_verginica,0]

print(f'f_oneway(_setosa_data, _vergicolor_data, _verginica_data):{f_oneway(_setosa_data, _vergicolor_data, _verginica_data)}')

F = f_oneway(_setosa_data, _vergicolor_data, _verginica_data)
print(f'F.statistic:{F.statistic:.3f}')
print(f'F.pvalue:{F.pvalue:.03f}')

# Titanic 号生存者解析

In [ ]:
from sklearn.datasets import fetch_openml

titanic = fetch_openml(data_id=40945, as_frame=True)
titanic.data

In [55]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,
                                       n_estimators=100, oob_score=True)

### 属性情報<!-- ### Attribute information  -->

* 抽出したデータセットの変数は，pclass, survived, name, age, embarked, home.dest, room, ticket, boat, sex:
* pclass:  乗客の階級 (1st, 2nd, 3rd) 社会経済階級の指標となる。
* age: 年齢，端数のある幼児もいる。
<!-- titanic2 のデータフレームには欠損データはなく，乗組員の記録も含まれているが，年齢は成人か子供かで二分されている。
これらのデータは Saint Mary's University の Robert Dawson から E-mail で入手した。
変数は pclass, age, sex, survived である。 -->
<!-- これらのデータ・フレームは，Hmisc の多くの関数のデモンストレーションや，Design ライブラリを使ったバイナリー・ロジスティック回帰分析のデモンストレーションに有用である。
詳細と参考文献は Simonoff, Jeffrey S (1997)を参照： 「"Unusual epsisode" and a second statistics course.  J Statistics Education, Vol.5 No.1. -->

<!--The variables on our extracted dataset are pclass, survived, name, age, embarked, home.dest, room, ticket, boat, and sex. pclass refers to passenger class (1st, 2nd, 3rd), and is a proxy for socio-economic class.
Age is in years, and some infants had fractional values.
The titanic2 data frame has no missing data and includes records for the crew, but age is dichotomized at adult vs. child.
These data were obtained from Robert Dawson, Saint Mary's University, E-mail.
The variables are pclass, age, sex, survived.
These data frames are useful for demonstrating many of the functions in Hmisc as well as demonstrating binary logistic regression analysis using the Design library.
For more details and references see Simonoff, Jeffrey S (1997): The "unusual episode" and a second statistics course. J Statistics Education, Vol. 5 No. 1. -->


* Passenger ID to identifiy the passenger, numerical feature (Passenger ID/Ticket Number).
* Survived is our label, as we can see is a binary feature, 1 if survived and 0 otherwise.
* Pclass is the Ticket class (1 = 1st (Upper), 2 = 2nd (Middle), 3 = 3rd (lower))
* Age is the age in years

* sibsp: 乗船した兄弟姉妹/配偶者の数
* parch: 乗船した両親／子供の数

<!-- * sibsp is the number of siblings/spouses aboard the Titanic
* parch is the number of parents/children aboard the Titanic-->

* ticket is the ticket number
* fare is the Passenger fare
* cabin is the cabin number
* embarked means Port of Embarkation. C = Cherbourg, Q = Queenstown, S = Southampton

<!-- https://github.com/Charlie5DH/Kaggle_Competitions/blob/master/Titanic_Machine_Learning_from_Disaster/titanic-disaster-cr.ipynb -->

In [ ]:
t2 = titanic.frame.rename(
    columns={'sex':'性別',
             'sibsp':'同乗兄弟姉妹数',
             'parch':'同乗両親子供数',
             'embarked':'乗船地',
             'pclass':'等級',
             'survived':'生死',
             'name':'氏名',
             'age':'年齢',
             'home.dest':'住所.目的地',
             'ticket':'チケット番号',
             'fare':'料金',
             'cabin':'客室番号',
             })
t2 = t2.drop(columns=['boat','body','住所.目的地','チケット番号','氏名','客室番号'])

t2['isFemale'] = [1 if x == 'female' else 0 for x in t2['性別']]
#X = t2.drop(columns=['生死']).to_numpy()
X = t2.drop(columns=['生死','性別','乗船地', '性別']).to_numpy()
y = t2['生死'].to_numpy()
print(X.shape, y.shape)
t2

In [59]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,
                                       n_estimators=100, oob_score=True)

# now lets split the data into train and test
from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((916, 6), (393, 6))

In [60]:
%%time
classifier_rf.fit(X_train, y_train)

# checking the oob score
classifier_rf.oob_score_

CPU times: user 190 ms, sys: 23.3 ms, total: 213 ms
Wall time: 188 ms


0.8133187772925764

In [61]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
params = {
    #'max_depth': [2,3,5,10,20],
    'max_depth': [2,3,4],
    #'min_samples_leaf': [5,10,20,50,100,200],
    'min_samples_leaf': [5,10,20],
    #'n_estimators': [10,25,30,50,100,200]
    'n_estimators': [10,25]
}

from sklearn.model_selection import GridSearchCV
# Instantiate the grid search model

grid_search = GridSearchCV(estimator=rf,
                           param_grid=params,
                           cv = 4,
                           n_jobs=-1, verbose=1, scoring="accuracy")

In [ ]:
%%time
grid_search.fit(X_train, y_train)

In [63]:
grid_search.best_score_

np.float64(0.7903930131004366)

In [ ]:
rf_best = grid_search.best_estimator_
rf_best

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(80,40))
plot_tree(rf_best.estimators_[5], feature_names = ['等級','年齢','同乗兄弟姉妹数','同乗両親子供数','料金','女性？'],class_names=['生','死'],filled=True);